In [30]:
%reload_ext autoreload
%autoreload 2
import sys, os
if 'google.colab' in sys.modules:
    GIT_ROOT = 'https://github.com/blueprints-for-text-analytics-python/early-release/raw/master'
    os.system(f'wget {GIT_ROOT}/ch05/setup.py')
import setup as conf

You are working on a local system.
Files will be searched relative to "..".
You are working on a local system.
Files will be searched relative to "..".


# Data preparation

In [3]:
sentences = ["It was the best of times", 
             "it was the worst of times", 
             "it was the age of wisdom", 
             "it was the age of foolishness"]

tokenized_sentences = [[t for t in sentence.split()] for sentence in sentences]

vocabulary = set([w for s in tokenized_sentences for w in s])

import pandas as pd
[[w, i] for i,w in enumerate(vocabulary)]

[['age', 0],
 ['was', 1],
 ['worst', 2],
 ['times', 3],
 ['It', 4],
 ['it', 5],
 ['best', 6],
 ['foolishness', 7],
 ['wisdom', 8],
 ['of', 9],
 ['the', 10]]

# One-hot by hand

In [4]:
def onehot_encode(tokenized_sentence):
    return [1 if w in tokenized_sentence else 0 for w in vocabulary]

onehot = [onehot_encode(tokenized_sentence) for tokenized_sentence in tokenized_sentences]

for (sentence, oh) in zip(sentences, onehot):
    print("%s: %s" % (oh, sentence))

[0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1]: It was the best of times
[0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1]: it was the worst of times
[1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1]: it was the age of wisdom
[1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1]: it was the age of foolishness


In [5]:
pd.DataFrame(onehot, columns=vocabulary)

,age,was,worst,times,It,it,best,foolishness,wisdom,of,the
0,0,1,0,1,1,0,1,0,0,1,1
1,0,1,1,1,0,1,0,0,0,1,1
2,1,1,0,0,0,1,0,0,1,1,1
3,1,1,0,0,0,1,0,1,0,1,1


In [6]:
sim = [onehot[0][i] & onehot[1][i] for i in range(0, len(vocabulary))]
sum(sim)

4

In [7]:
import numpy as np
np.dot(onehot[0], onehot[1])

4

In [8]:
np.dot(onehot, onehot[1])

array([4, 6, 4, 4])

## Out of vocabulary

In [9]:
onehot_encode("the age of wisdom is the best of times".split())

[1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1]

In [10]:
onehot_encode("John likes to watch movies. Mary likes movies too.".split())

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

## document term matrix

In [11]:
onehot

[[0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1],
 [0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1],
 [1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1],
 [1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1]]

## similarities

In [12]:
import numpy as np
np.dot(onehot, np.transpose(onehot))

array([[6, 4, 3, 3],
       [4, 6, 4, 4],
       [3, 4, 6, 5],
       [3, 4, 5, 6]])

# scikit learn one-hot vectorization

In [13]:
from sklearn.preprocessing import MultiLabelBinarizer
lb = MultiLabelBinarizer()
lb.fit([vocabulary])
lb.transform(tokenized_sentences)

array([[1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1],
       [0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0],
       [0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0]])

# CountVectorizer

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [15]:
more_sentences = sentences + ["John likes to watch movies. Mary likes movies too.",
                              "Mary also likes to watch football games."]
pd.DataFrame(more_sentences)

,0
0,It was the best of times
1,it was the worst of times
2,it was the age of wisdom
3,it was the age of foolishness
4,John likes to watch movies. Mary likes movies ...
5,Mary also likes to watch football games.


In [16]:
cv.fit(more_sentences)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [17]:
print(cv.get_feature_names())

['age', 'also', 'best', 'foolishness', 'football', 'games', 'it', 'john', 'likes', 'mary', 'movies', 'of', 'the', 'times', 'to', 'too', 'was', 'watch', 'wisdom', 'worst']


In [18]:
dt = cv.transform(more_sentences)

In [19]:
dt

<6x20 sparse matrix of type '<class 'numpy.int64'>'
	with 38 stored elements in Compressed Sparse Row format>

In [20]:
pd.DataFrame(dt.toarray(), columns=cv.get_feature_names())

,age,also,best,foolishness,football,games,it,john,likes,mary,movies,of,the,times,to,too,was,watch,wisdom,worst
0,0,0,1,0,0,0,1,0,0,0,0,1,1,1,0,0,1,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0,1,1,1,0,0,1,0,0,1
2,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,1,0
3,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,1,2,1,2,0,0,0,1,1,0,1,0,0
5,0,1,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,1,0,0


In [21]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(dt[0], dt[1])

array([[0.83333333]])

In [22]:
len(more_sentences)

6

In [23]:
pd.DataFrame(cosine_similarity(dt, dt))

,0,1,2,3,4,5
0,1.000000,0.833333,0.666667,0.666667,0.000000,0.000000
1,0.833333,1.000000,0.666667,0.666667,0.000000,0.000000
2,0.666667,0.666667,1.000000,0.833333,0.000000,0.000000
3,0.666667,0.666667,0.833333,1.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,1.000000,0.524142
5,0.000000,0.000000,0.000000,0.000000,0.524142,1.000000


# TF/IDF

In [24]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
tfidf_dt = tfidf.fit_transform(dt)

In [25]:
pd.DataFrame(tfidf_dt.toarray(), columns=cv.get_feature_names())

,age,also,best,foolishness,football,games,it,john,likes,mary,movies,of,the,times,to,too,was,watch,wisdom,worst
0,0.000000,0.000000,0.56978,0.00000,0.000000,0.000000,0.338027,0.000000,0.000000,0.000000,0.000000,0.338027,0.338027,0.467228,0.000000,0.000000,0.338027,0.000000,0.00000,0.00000
1,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.338027,0.000000,0.000000,0.000000,0.000000,0.338027,0.338027,0.467228,0.000000,0.000000,0.338027,0.000000,0.00000,0.56978
2,0.467228,0.000000,0.00000,0.00000,0.000000,0.000000,0.338027,0.000000,0.000000,0.000000,0.000000,0.338027,0.338027,0.000000,0.000000,0.000000,0.338027,0.000000,0.56978,0.00000
3,0.467228,0.000000,0.00000,0.56978,0.000000,0.000000,0.338027,0.000000,0.000000,0.000000,0.000000,0.338027,0.338027,0.000000,0.000000,0.000000,0.338027,0.000000,0.00000,0.00000
4,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.305609,0.501208,0.250604,0.611219,0.000000,0.000000,0.000000,0.250604,0.305609,0.000000,0.250604,0.00000,0.00000
5,0.000000,0.419233,0.00000,0.00000,0.419233,0.419233,0.000000,0.000000,0.343777,0.343777,0.000000,0.000000,0.000000,0.000000,0.343777,0.000000,0.000000,0.343777,0.00000,0.00000


In [26]:
pd.DataFrame(cosine_similarity(tfidf_dt, tfidf_dt))

,0,1,2,3,4,5
0,1.000000,0.675351,0.457049,0.457049,0.00000,0.00000
1,0.675351,1.000000,0.457049,0.457049,0.00000,0.00000
2,0.457049,0.457049,1.000000,0.675351,0.00000,0.00000
3,0.457049,0.457049,0.675351,1.000000,0.00000,0.00000
4,0.000000,0.000000,0.000000,0.000000,1.00000,0.43076
5,0.000000,0.000000,0.000000,0.000000,0.43076,1.00000


In [31]:
headlines = pd.read_csv(conf.ABCNEWS_FILE, parse_dates=["publish_date"])
headlines.head()

,publish_date,headline_text
0,2003-02-19,aba decides against community broadcasting lic...
1,2003-02-19,act fire witnesses must be aware of defamation
2,2003-02-19,a g calls for infrastructure protection summit
3,2003-02-19,air nz staff in aust strike for pay rise
4,2003-02-19,air nz strike to affect australian travellers


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
dt = tfidf.fit_transform(headlines["headline_text"])

In [33]:
dt

<1103663x95878 sparse matrix of type '<class 'numpy.float64'>'
	with 7001357 stored elements in Compressed Sparse Row format>

In [34]:
dt.data.nbytes

56010856

In [35]:
%%time
cosine_similarity(dt[0:10000], dt[0:10000])

CPU times: user 108 ms, sys: 225 ms, total: 333 ms
Wall time: 333 ms


array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.16913596,
        0.16792138],
       [0.        , 0.        , 0.        , ..., 0.16913596, 1.        ,
        0.33258708],
       [0.        , 0.        , 0.        , ..., 0.16792138, 0.33258708,
        1.        ]])

## Stopwords

In [36]:
from spacy.lang.en.stop_words import STOP_WORDS as stopwords
print(len(stopwords))
tfidf = TfidfVectorizer(stop_words=stopwords)
dt = tfidf.fit_transform(headlines["headline_text"])
dt

326


/home/cwinkler/.venv/jupyter/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:297: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


<1103663x95600 sparse matrix of type '<class 'numpy.float64'>'
	with 5644186 stored elements in Compressed Sparse Row format>

## min_df

In [37]:
tfidf = TfidfVectorizer(stop_words=stopwords, min_df=2)
dt = tfidf.fit_transform(headlines["headline_text"])
dt

/home/cwinkler/.venv/jupyter/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:297: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


<1103663x58527 sparse matrix of type '<class 'numpy.float64'>'
	with 5607113 stored elements in Compressed Sparse Row format>

In [38]:
tfidf = TfidfVectorizer(stop_words=stopwords, min_df=.0001)
dt = tfidf.fit_transform(headlines["headline_text"])
dt

/home/cwinkler/.venv/jupyter/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:297: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


<1103663x6772 sparse matrix of type '<class 'numpy.float64'>'
	with 4816381 stored elements in Compressed Sparse Row format>

## max_df

In [39]:
tfidf = TfidfVectorizer(stop_words=stopwords, max_df=0.1)
dt = tfidf.fit_transform(headlines["headline_text"])
dt

/home/cwinkler/.venv/jupyter/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:297: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


<1103663x95600 sparse matrix of type '<class 'numpy.float64'>'
	with 5644186 stored elements in Compressed Sparse Row format>

In [40]:
tfidf = TfidfVectorizer(max_df=0.1)
dt = tfidf.fit_transform(headlines["headline_text"])
dt

<1103663x95875 sparse matrix of type '<class 'numpy.float64'>'
	with 6532752 stored elements in Compressed Sparse Row format>

## n-grams

In [41]:
tfidf = TfidfVectorizer(stop_words=stopwords, ngram_range=(1,2), min_df=2)
dt = tfidf.fit_transform(headlines["headline_text"])
print(dt.shape)
print(dt.data.nbytes)
tfidf = TfidfVectorizer(stop_words=stopwords, ngram_range=(1,3), min_df=2)
dt = tfidf.fit_transform(headlines["headline_text"])
print(dt.shape)
print(dt.data.nbytes)

/home/cwinkler/.venv/jupyter/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:297: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


(1103663, 559961)
67325400


/home/cwinkler/.venv/jupyter/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:297: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


(1103663, 747988)
72360104


## Lemmas

In [43]:
from tqdm.auto import tqdm
import spacy
nlp = spacy.load("en")
nouns_adjectives_verbs = ["NOUN", "PROPN", "ADJ", "ADV", "VERB"]
for i, row in tqdm(headlines.iterrows(), total=len(headlines)):
    doc = nlp(str(row["headline_text"]))
    headlines.at[i, "lemmas"] = " ".join([token.lemma_ for token in doc])
    headlines.at[i, "nav"] = " ".join([token.lemma_ for token in doc if token.pos_ in nouns_adjectives_verbs])

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

In [ ]:
headlines.head()

In [ ]:
tfidf = TfidfVectorizer(stop_words=stopwords)
dt = tfidf.fit_transform(headlines["lemmas"].map(str))
dt

In [ ]:
tfidf = TfidfVectorizer(stop_words=stopwords)
dt = tfidf.fit_transform(headlines["nav"].map(str))
dt

## remove top 10,000

In [ ]:
top_10000 = pd.read_csv("https://raw.githubusercontent.com/first20hours/google-10000-english/master/google-10000-english.txt", header=None)
tfidf = TfidfVectorizer(stop_words=set(top_10000.iloc[:,0].values))
dt = tfidf.fit_transform(headlines["nav"].map(str))
dt

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,2), stop_words=set(top_10000.iloc[:,0].values), min_df=2)
dt = tfidf.fit_transform(headlines["nav"].map(str))
dt

## Finding document most similar to made-up document

In [ ]:
tfidf = TfidfVectorizer(stop_words=stopwords, min_df=2)
dt = tfidf.fit_transform(headlines["lemmas"].map(str))
dt

In [ ]:
made_up = tfidf.transform(["australia and new zealand discuss optimal apple size"])

In [ ]:
sim = cosine_similarity(made_up, dt)

In [ ]:
sim[0]

In [ ]:
headlines.iloc[np.argsort(sim[0])[::-1][0:5]][["publish_date", "lemmas"]]

# Finding the most similar documents

In [ ]:
# there are "test" headlines in the corpus
stopwords.add("test")
tfidf = TfidfVectorizer(stop_words=stopwords, ngram_range=(1,2), min_df=2, norm='l2')
dt = tfidf.fit_transform(headlines["headline_text"])

### Timing Cosine Similarity

In [ ]:
%%time
cosine_similarity(dt[0:10000], dt[0:10000], dense_output=False)

In [ ]:
%%time
r = cosine_similarity(dt[0:10000], dt[0:10000])
r[r > 0.9999] = 0
print(np.argmax(r))

In [ ]:
%%time
r = cosine_similarity(dt[0:10000], dt[0:10000], dense_output=False)
r[r > 0.9999] = 0
print(np.argmax(r))

### Timing Dot-Product

In [ ]:
%%time
r = np.dot(dt[0:10000], np.transpose(dt[0:10000]))
r[r > 0.9999] = 0
print(np.argmax(r))

## Batch

In [ ]:
%%time
batch = 10000
max_sim = 0.0
max_a = None
max_b = None
for a in range(0, dt.shape[0], batch):
    for b in range(0, a+batch, batch):
        print(a, b)
        #r = np.dot(dt[a:a+batch], np.transpose(dt[b:b+batch]))
        r = cosine_similarity(dt[a:a+batch], dt[b:b+batch], dense_output=False)
        # eliminate identical vectors
        # by setting their similarity to np.nan which gets sorted out
        r[r > 0.9999] = 0
        sim = r.max()
        if sim > max_sim:
            # argmax returns a single value which we have to 
            # map to the two dimensions            
            (max_a, max_b) = np.unravel_index(np.argmax(r), r.shape)
            # adjust offsets in corpus (this is a submatrix)
            max_a += a
            max_b += b
            max_sim = sim

In [ ]:
print(max_a, max_b)

In [ ]:
print(max_sim)

In [ ]:
pd.set_option('max_colwidth', -1)
headlines.iloc[[max_a, max_b]][["publish_date", "headline_text"]]

# Finding most related words

In [ ]:
tfidf_word = TfidfVectorizer(stop_words=stopwords, min_df=1000)
dt_word = tfidf_word.fit_transform(headlines["headline_text"])

In [ ]:
r = cosine_similarity(dt_word.T, dt_word.T)
np.fill_diagonal(r, 0)

In [ ]:
voc = tfidf_word.get_feature_names()
size = r.shape[0] # quadratic
for index in np.argsort(r.flatten())[::-1][0:40]:
    a = int(index/size)
    b = index%size
    if a > b:  # avoid repetitions
        print('"%s" related to "%s"' % (voc[a], voc[b]))